In [7]:
import pandas as pandas

NOME_DO_ARQUIVO = 'noticias_estadao.csv'
indice_invertido = {}

# olhar a biblioteca nltk

dados = pandas.read_csv(NOME_DO_ARQUIVO)
for i in range(dados.titulo.size):
    texto = dados.titulo[i] + " " + dados.conteudo[i]
    palavras_divididas = texto.split(" ")
    for palavra in palavras_divididas:
        if palavra.lower() not in indice_invertido:
            indice_invertido[palavra.lower()] = [dados.idNoticia[i]]
        else:
            indice_invertido[palavra.lower()].append(dados.idNoticia[i])

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

campina = indice_invertido["Campina".lower()]
grande = indice_invertido["Grande".lower()]

intersecao = intersection(campina, grande)
print(intersecao)

[1068, 1370, 1770, 1952, 1952, 1952, 1987, 2779, 4802, 4802, 5870, 6694]
